In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow import keras
import tensorflow as tf
import csv
from sklearn.decomposition import PCA
import sklearn
import pickle
from keras.layers import LSTM, Dense, SimpleRNN
import matplotlib.pyplot as plt
import statistics
from sklearn.model_selection import train_test_split
from keras.layers import Dense,Dropout,Conv3D,Conv1D,Conv2D,Flatten,MaxPool2D
from sklearn.metrics import accuracy_score
%load_ext tensorboard

In [2]:
import glob
files = glob.glob("../../datasets/prop-1.csv", recursive = True)
ant_master = pd.concat(map(pd.read_csv, files))

In [3]:
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom  ca  ce  ...       dam  \
0      prop-1        4    2    4    0    2    5     1   0   2  ...  1.000000   
1      prop-1        4   12    4    1   19   44    66   2  18  ...  0.000000   
2      prop-1        4   10    4    0   13   48    45   4  13  ...  1.000000   
3      prop-1        4    2    3    0    5    7     1   0   5  ...  1.000000   
4      prop-1        4    8    1    0    4   38     0   3   1  ...  0.500000   
...       ...      ...  ...  ...  ...  ...  ...   ...  ..  ..  ...       ...   
18466  prop-1      185   17    1    0   24   67    96   9  15  ...  0.150000   
18467  prop-1      185    9    1    0   25   49    14   7  18  ...  0.333333   
18468  prop-1      185    6    3    0   10   29     0   0  10  ...  0.800000   
18469  prop-1      185    5    5    0   28    9    10  26   2  ...  1.000000   
18470  prop-1      185   11    1    0   54   34     9  51   3  ...  0.666667   

       moa       mfa       cam  ic  cbm        amc  max_cc  avg_cc  bug  
0        0  0.941176  0.750000   1    1  10.500000       3  1.5000    0  
1        0  0.905172  0.350000   2    3  20.583333       8  2.0833    0  
2        0  0.918919  0.500000   2    3  19.100000       4  1.2000    0  
3        0  0.933333  0.750000   1    1  11.500000       4  2.0000    1  
4        0  0.000000  0.285714   0    0  30.625000       2  0.8750    0  
...    ...       ...       ...  ..  ...        ...     ...     ...  ...  
18466    0  0.000000  0.250000   0    0  41.529412       6  2.6471    1  
18467    0  0.000000  0.319444   0    0  29.888889       3  1.3333    0  
18468    0  0.818182  0.400000   0    0  45.500000       6  2.1667    0  
18469    0  0.962963  0.650000   1    1   4.400000       1  0.2000    0  
18470    1  0.000000  0.400000   0    0  25.545455       6  1.9091    0  

[18471 rows x 23 columns]

In [4]:
ant_master = ant_master.loc[ant_master['version'] == 85]
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom  ca  ce  ...       dam  \
7075   prop-1       85    4    3    0   11   12     0   3   8  ...  1.000000   
7076   prop-1       85    0    1    0    0    0     0   0   0  ...  0.000000   
7077   prop-1       85    2    2    0    3    4     1   0   3  ...  0.000000   
7078   prop-1       85   55    3   10   72  218  1263  23  57  ...  0.217391   
7079   prop-1       85    2    1    0   10   25     0   1   9  ...  1.000000   
...       ...      ...  ...  ...  ...  ...  ...   ...  ..  ..  ...       ...   
10147  prop-1       85    5    5    0    3    9    10   1   2  ...  1.000000   
10148  prop-1       85    5    5    0    9    9    10   7   2  ...  1.000000   
10149  prop-1       85    6    3    0   27   50     3   0  27  ...  0.625000   
10150  prop-1       85    5    1    0   12   25     8   1  11  ...  0.500000   
10151  prop-1       85    2    3    0    8   20     1   0   8  ...  0.000000   

       moa       mfa       cam  ic  cbm        amc  max_cc  avg_cc  bug  
7075     1  0.812500  0.500000   0    0  23.000000       1  0.7500    2  
7076     0  0.000000  0.000000   0    0   0.000000       0  0.0000    0  
7077     0  0.941176  0.666667   1    1   4.000000       1  0.5000    0  
7078     2  0.442105  0.160494   1    5  26.854545      25  2.2727    8  
7079     0  0.000000  0.666667   0    0  42.000000       5  2.5000    0  
...    ...       ...       ...  ..  ...        ...     ...     ...  ...  
10147    0  0.962963  0.650000   1    1   4.400000       1  0.2000    0  
10148    0  0.962963  0.650000   1    1   4.400000       1  0.2000    0  
10149    1  0.807692  0.416667   0    0  57.166667       4  2.0000    1  
10150    0  0.000000  0.750000   0    0  16.200000       2  0.8000    0  
10151    0  0.875000  0.666667   1    1  46.000000       5  2.5000    1  

[3077 rows x 23 columns]

In [5]:
train, test = train_test_split(ant_master, test_size=0.2)

In [6]:
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
x_train = train[cols_to_norm]
x_test = test[cols_to_norm]

In [7]:
x_train

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3  loc       dam  \
7555    1    1    0    0    5     0   0   0    0  2.000000   18  0.000000   
9013    4    3    0   14   27     4   0  14    2  0.833333  113  0.500000   
7111    2    3    0    1    7     1   0   1    1  2.000000   42  1.000000   
7855    1    1    0    4    1     0   0   4    1  2.000000    1  0.000000   
9373    2    2    0   16   15     1   0  16    2  2.000000   55  0.000000   
...   ...  ...  ...  ...  ...   ...  ..  ..  ...       ...  ...       ...   
9703    2    2    0   13   14     1   0  13    2  1.000000   49  0.000000   
8227    2    1    0   13   23     1   0  13    2  1.500000   82  0.000000   
7970    2    2    0    2    3     1   1   1    1  2.000000    7  0.000000   
7228    4    1    0    7    4     6   5   2    4  2.000000    4  0.000000   
9249    8    1    0    6   52     0   1   5    2  0.870130  528  0.909091   

      moa       mfa       cam  ic  cbm     amc  max_cc  avg_cc  
7555    0  0.000000  0.000000   0    0  12.000       0   0.000  
9013    0  0.777778  0.666667   1    1  26.750       5   1.750  
7111    0  0.980000  0.666667   0    0  19.500       1   0.500  
7855    0  0.000000  1.000000   0    0   0.000       1   1.000  
9373    0  0.800000  0.666667   1    1  26.500       1   0.500  
...   ...       ...       ...  ..  ...     ...     ...     ...  
9703    0  0.800000  0.666667   1    1  23.000       1   0.500  
8227    0  0.000000  0.666667   0    0  39.000       3   1.500  
7970    0  0.909091  1.000000   1    2   2.500       1   0.500  
7228    0  0.000000  0.500000   0    0   0.000       1   1.000  
9249    1  0.000000  0.367347   0    0  63.625       4   2.375  

[2461 rows x 20 columns]

In [8]:
x_test

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3   loc       dam  \
7607     1    1    0   20    1     0  19   1    1  2.000000     1  0.000000   
7187     3    1    0    8    6     3   1   7    3  2.000000    22  0.000000   
9521     6    3    0    6   16    13   0   6    1  0.900000    92  0.500000   
7587     6    3    0   12   29     9   0  12    6  0.700000   149  0.500000   
10085   32    1    0    8   33   492   0   8   32  0.967742    97  1.000000   
...    ...  ...  ...  ...  ...   ...  ..  ..  ...       ...   ...       ...   
9550    23    3    0   26   77   119   0  26   13  0.886364  1666  0.875000   
9884     9    1    0   25   50    14   7  18    7  0.666667   282  0.333333   
7818     1    1    0    0    5     0   0   0    0  2.000000    15  0.000000   
8327     3    2    0   15   20     3   0  15    1  0.500000    73  0.000000   
8902     3    3    0    8   14     1   0   8    2  0.833333    65  0.333333   

       moa       mfa       cam  ic  cbm        amc  max_cc  avg_cc  
7607     0  0.000000  1.000000   0    0   0.000000       1  1.0000  
7187     0  0.000000  0.733333   0    0   6.333333       1  0.6667  
9521     0  0.789474  0.560000   1    4  14.000000       5  1.3333  
7587     0  0.886364  0.333333   2    6  23.500000      15  3.5000  
10085    0  0.000000  0.130682   0    0   1.968750       1  0.9688  
...    ...       ...       ...  ..  ...        ...     ...     ...  
9550     0  0.461538  0.295455   0    0  70.739130      13  3.1739  
9884     0  0.000000  0.319444   0    0  30.000000       3  1.3333  
7818     0  0.000000  0.000000   0    0  12.000000       0  0.0000  
8327     0  0.600000  0.533333   0    0  23.000000       1  0.6667  
8902     0  0.875000  0.666667   1    1  19.666667       3  1.0000  

[616 rows x 20 columns]

In [9]:
y_train = pd.DataFrame(train['bug'])
y_test = pd.DataFrame(test['bug'])

In [10]:
y_train

bug
7555    1
9013    0
7111    0
7855    0
9373    1
...   ...
9703    1
8227    0
7970    0
7228    0
9249    0

[2461 rows x 1 columns]

In [11]:
y_test

bug
7607     0
7187     0
9521     1
7587     1
10085    0
...    ...
9550     1
9884     1
7818     1
8327     0
8902     1

[616 rows x 1 columns]

In [12]:
#Getting the rows and columns size in our data
img_rows, img_cols = 1,20
# Preparing the data for the model.
X_train_matrix = x_train.values
X_test_matrix = x_test.values
Y_train_matrix = y_train.values
Y_test_matrix = y_test.values
# Preparing the data for the model.
Ytrainseries = y_train['bug']
Ytestseries = y_test['bug']

In [13]:
# Preparing the data for the model.
X_train1 = X_train_matrix.reshape(X_train_matrix.shape[0], img_rows, img_cols, 1)
X_test1 = X_test_matrix.reshape(X_test_matrix.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

In [14]:
def define_model():
    # Designing and initializing the model.
    #Building the model
    model = Sequential()
    #add model layers
    model.add(Conv1D(2, kernel_size=2, activation='relu'))
    model.add(tf.keras.layers.Flatten())
    model.add(Dense(1, activation='relu'))
    #compile model using mse as the loss function
    model.compile(loss='mse', optimizer = keras.optimizers.Adam(learning_rate=0.0005), metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()])
    return model

In [15]:
y = y_test['bug'].tolist()
y

[0,
 0,
 1,
 1,
 0,
 0,
 0,
 3,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 5,
 1,
 0,
 0,
 3,
 0,
 0,
 1,
 1,
 2,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 2,
 0,
 3,
 5,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 4,
 5,
 0,
 3,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 4,
 0,
 0,
 0,
 1,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 3,
 0,
 0,
 1,
 1,
 0,
 3,
 0,
 0,
 0,
 1,
 0,
 3,
 0,
 3,
 1,
 1,
 0,
 1,
 0,
 3,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 2,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 3,
 2,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 1,
 4,
 1,
 0,
 0,
 0,
 0,
 0,
 3,
 1,
 2,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,


In [16]:
total_training_time = []
total_scores = []
total_training_loss_history = []
test_ARE = []
for i in range(30):
    print("Run no: " + str(i))
    model = define_model()
    log_dir = '/home/bavanya/Documents/Training_curves/cnn_propv85/' + 'run' + str(i)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    start = time.time()
    history = model.fit(X_train1, Y_train_matrix, epochs = 100, batch_size = 100, callbacks=[tensorboard_callback])
    stop = time.time()
    total_training_time.append(stop - start) 
    total_training_loss_history.append(history.history)
    score = model.evaluate(X_test1, Y_test_matrix)
    total_scores.append(score) 
    predictions = model.predict(X_test1)
    x = predictions.tolist()
    x = list(np.concatenate(x))
    x = [int(i) for i in x]
    ARE_test = tf.keras.metrics.MeanRelativeError(normalizer=y)
    ARE_test.update_state(y, x)
    test_ARE.append(ARE_test.result().numpy())
    print("score of " + str(i) + "th run is: " + ', '.join(map(str,score)))

Run no: 0
Epoch 1/100
25/25 [==============================] - 0s 2ms/step - loss: 20727.8379 - mse: 20727.8379 - mae: 64.0515 - root_mean_squared_error: 143.9716 - mean_squared_logarithmic_error: 10.2628
Epoch 2/100
25/25 [==============================] - 0s 1ms/step - loss: 17949.1367 - mse: 17949.1367 - mae: 59.0894 - root_mean_squared_error: 133.9744 - mean_squared_logarithmic_error: 9.7725
Epoch 3/100
25/25 [==============================] - 0s 1ms/step - loss: 15661.0693 - mse: 15661.0693 - mae: 54.4788 - root_mean_squared_error: 125.1442 - mean_squared_logarithmic_error: 9.2938
Epoch 4/100
25/25 [==============================] - 0s 1ms/step - loss: 13645.5557 - mse: 13645.5557 - mae: 50.2773 - root_mean_squared_error: 116.8142 - mean_squared_logarithmic_error: 8.8471
Epoch 5/100
25/25 [==============================] - 0s 1ms/step - loss: 11973.6670 - mse: 11973.6670 - mae: 46.5042 - root_mean_squared_error: 109.4243 - mean_squared_logarithmic_error: 8.4105
Epoch 6/100
25/25 [

25/25 [==============================] - 0s 1ms/step - loss: 43.0379 - mse: 43.0379 - mae: 1.1274 - root_mean_squared_error: 6.5603 - mean_squared_logarithmic_error: 0.5248
Epoch 86/100
25/25 [==============================] - 0s 1ms/step - loss: 41.3027 - mse: 41.3027 - mae: 1.1139 - root_mean_squared_error: 6.4267 - mean_squared_logarithmic_error: 0.5221
Epoch 87/100
25/25 [==============================] - 0s 1ms/step - loss: 39.3343 - mse: 39.3343 - mae: 1.1047 - root_mean_squared_error: 6.2717 - mean_squared_logarithmic_error: 0.5201
Epoch 88/100
25/25 [==============================] - 0s 1ms/step - loss: 38.3174 - mse: 38.3174 - mae: 1.0948 - root_mean_squared_error: 6.1901 - mean_squared_logarithmic_error: 0.5177
Epoch 89/100
25/25 [==============================] - 0s 1ms/step - loss: 36.1525 - mse: 36.1525 - mae: 1.0852 - root_mean_squared_error: 6.0127 - mean_squared_logarithmic_error: 0.5166
Epoch 90/100
25/25 [==============================] - 0s 1ms/step - loss: 34.7313 -

25/25 [==============================] - 0s 1ms/step - loss: 66.4421 - mse: 66.4421 - mae: 1.0827 - root_mean_squared_error: 8.1512 - mean_squared_logarithmic_error: 0.5019
Epoch 69/100
25/25 [==============================] - 0s 1ms/step - loss: 61.7108 - mse: 61.7108 - mae: 1.0708 - root_mean_squared_error: 7.8556 - mean_squared_logarithmic_error: 0.5008
Epoch 70/100
25/25 [==============================] - 0s 1ms/step - loss: 57.8855 - mse: 57.8855 - mae: 1.0605 - root_mean_squared_error: 7.6083 - mean_squared_logarithmic_error: 0.4999
Epoch 71/100
25/25 [==============================] - 0s 1ms/step - loss: 53.4767 - mse: 53.4767 - mae: 1.0503 - root_mean_squared_error: 7.3128 - mean_squared_logarithmic_error: 0.4989
Epoch 72/100
25/25 [==============================] - 0s 1ms/step - loss: 51.3808 - mse: 51.3808 - mae: 1.0428 - root_mean_squared_error: 7.1680 - mean_squared_logarithmic_error: 0.4981
Epoch 73/100
25/25 [==============================] - 0s 1ms/step - loss: 46.7117 -

25/25 [==============================] - 0s 1ms/step - loss: 5.5889 - mse: 5.5889 - mae: 0.8233 - root_mean_squared_error: 2.3641 - mean_squared_logarithmic_error: 0.4653
Epoch 54/100
25/25 [==============================] - 0s 1ms/step - loss: 5.5774 - mse: 5.5774 - mae: 0.8228 - root_mean_squared_error: 2.3616 - mean_squared_logarithmic_error: 0.4651
Epoch 55/100
25/25 [==============================] - 0s 1ms/step - loss: 5.5670 - mse: 5.5670 - mae: 0.8224 - root_mean_squared_error: 2.3594 - mean_squared_logarithmic_error: 0.4649
Epoch 56/100
25/25 [==============================] - 0s 1ms/step - loss: 5.5591 - mse: 5.5591 - mae: 0.8220 - root_mean_squared_error: 2.3578 - mean_squared_logarithmic_error: 0.4648
Epoch 57/100
25/25 [==============================] - 0s 1ms/step - loss: 5.5496 - mse: 5.5496 - mae: 0.8215 - root_mean_squared_error: 2.3557 - mean_squared_logarithmic_error: 0.4645
Epoch 58/100
25/25 [==============================] - 0s 1ms/step - loss: 5.5438 - mse: 5.543

25/25 [==============================] - 0s 1ms/step - loss: 23.1939 - mse: 23.1939 - mae: 1.9215 - root_mean_squared_error: 4.8160 - mean_squared_logarithmic_error: 0.8113
Epoch 38/100
25/25 [==============================] - 0s 1ms/step - loss: 21.4797 - mse: 21.4797 - mae: 1.8355 - root_mean_squared_error: 4.6346 - mean_squared_logarithmic_error: 0.7845
Epoch 39/100
25/25 [==============================] - 0s 998us/step - loss: 19.9181 - mse: 19.9181 - mae: 1.7588 - root_mean_squared_error: 4.4630 - mean_squared_logarithmic_error: 0.7599
Epoch 40/100
25/25 [==============================] - 0s 1ms/step - loss: 18.5001 - mse: 18.5001 - mae: 1.6962 - root_mean_squared_error: 4.3012 - mean_squared_logarithmic_error: 0.7367
Epoch 41/100
25/25 [==============================] - 0s 1ms/step - loss: 17.3868 - mse: 17.3868 - mae: 1.6419 - root_mean_squared_error: 4.1697 - mean_squared_logarithmic_error: 0.7183
Epoch 42/100
25/25 [==============================] - 0s 1ms/step - loss: 16.3759

...

25/25 [==============================] - 0s 1ms/step - loss: 5.5028 - mse: 5.5028 - mae: 0.8214 - root_mean_squared_error: 2.3458 - mean_squared_logarithmic_error: 0.4600
Epoch 92/100
25/25 [==============================] - 0s 951us/step - loss: 5.5028 - mse: 5.5028 - mae: 0.8218 - root_mean_squared_error: 2.3458 - mean_squared_logarithmic_error: 0.4601
Epoch 93/100
25/25 [==============================] - 0s 957us/step - loss: 5.5027 - mse: 5.5027 - mae: 0.8211 - root_mean_squared_error: 2.3458 - mean_squared_logarithmic_error: 0.4599
Epoch 94/100
25/25 [==============================] - 0s 1ms/step - loss: 5.5028 - mse: 5.5028 - mae: 0.8208 - root_mean_squared_error: 2.3458 - mean_squared_logarithmic_error: 0.4598
Epoch 95/100
25/25 [==============================] - 0s 1ms/step - loss: 5.5027 - mse: 5.5027 - mae: 0.8207 - root_mean_squared_error: 2.3458 - mean_squared_logarithmic_error: 0.4598
Epoch 96/100
25/25 [==============================] - 0s 979us/step - loss: 5.5027 - mse:

25/25 [==============================] - 0s 933us/step - loss: 5.5149 - mse: 5.5149 - mae: 0.8191 - root_mean_squared_error: 2.3484 - mean_squared_logarithmic_error: 0.4626
Epoch 75/100
25/25 [==============================] - 0s 1ms/step - loss: 5.5144 - mse: 5.5144 - mae: 0.8191 - root_mean_squared_error: 2.3483 - mean_squared_logarithmic_error: 0.4626
Epoch 76/100
25/25 [==============================] - 0s 1ms/step - loss: 5.5131 - mse: 5.5131 - mae: 0.8190 - root_mean_squared_error: 2.3480 - mean_squared_logarithmic_error: 0.4626
Epoch 77/100
25/25 [==============================] - 0s 975us/step - loss: 5.5130 - mse: 5.5130 - mae: 0.8189 - root_mean_squared_error: 2.3480 - mean_squared_logarithmic_error: 0.4627
Epoch 78/100
25/25 [==============================] - 0s 966us/step - loss: 5.5121 - mse: 5.5121 - mae: 0.8188 - root_mean_squared_error: 2.3478 - mean_squared_logarithmic_error: 0.4629
Epoch 79/100
25/25 [==============================] - 0s 1ms/step - loss: 5.5116 - mse:

25/25 [==============================] - 0s 887us/step - loss: 5.6755 - mse: 5.6755 - mae: 0.8866 - root_mean_squared_error: 2.3823 - mean_squared_logarithmic_error: 0.4872
Epoch 60/100
25/25 [==============================] - 0s 924us/step - loss: 5.6694 - mse: 5.6694 - mae: 0.8852 - root_mean_squared_error: 2.3810 - mean_squared_logarithmic_error: 0.4867
Epoch 61/100
25/25 [==============================] - 0s 948us/step - loss: 5.6642 - mse: 5.6642 - mae: 0.8837 - root_mean_squared_error: 2.3800 - mean_squared_logarithmic_error: 0.4861
Epoch 62/100
25/25 [==============================] - 0s 961us/step - loss: 5.6599 - mse: 5.6599 - mae: 0.8821 - root_mean_squared_error: 2.3791 - mean_squared_logarithmic_error: 0.4855
Epoch 63/100
25/25 [==============================] - 0s 891us/step - loss: 5.6572 - mse: 5.6572 - mae: 0.8804 - root_mean_squared_error: 2.3785 - mean_squared_logarithmic_error: 0.4851
Epoch 64/100
25/25 [==============================] - 0s 946us/step - loss: 5.6507 

Epoch 1/100
25/25 [==============================] - 0s 3ms/step - loss: 3447.2974 - mse: 3447.2974 - mae: 22.0762 - root_mean_squared_error: 58.7137 - mean_squared_logarithmic_error: 4.6874
Epoch 2/100
25/25 [==============================] - 0s 981us/step - loss: 2227.0635 - mse: 2227.0635 - mae: 16.7864 - root_mean_squared_error: 47.1918 - mean_squared_logarithmic_error: 3.8068
Epoch 3/100
25/25 [==============================] - 0s 939us/step - loss: 1385.5800 - mse: 1385.5800 - mae: 12.4715 - root_mean_squared_error: 37.2234 - mean_squared_logarithmic_error: 3.0344
Epoch 4/100
25/25 [==============================] - 0s 907us/step - loss: 838.2017 - mse: 838.2017 - mae: 9.0644 - root_mean_squared_error: 28.9517 - mean_squared_logarithmic_error: 2.3717
Epoch 5/100
25/25 [==============================] - 0s 942us/step - loss: 513.2931 - mse: 513.2931 - mae: 6.7210 - root_mean_squared_error: 22.6560 - mean_squared_logarithmic_error: 1.8924
Epoch 6/100
25/25 [========================

25/25 [==============================] - 0s 1ms/step - loss: 5.5825 - mse: 5.5825 - mae: 0.8249 - root_mean_squared_error: 2.3627 - mean_squared_logarithmic_error: 0.4635
Epoch 89/100
25/25 [==============================] - 0s 990us/step - loss: 5.5803 - mse: 5.5803 - mae: 0.8248 - root_mean_squared_error: 2.3623 - mean_squared_logarithmic_error: 0.4635
Epoch 90/100
25/25 [==============================] - 0s 938us/step - loss: 5.5773 - mse: 5.5773 - mae: 0.8246 - root_mean_squared_error: 2.3616 - mean_squared_logarithmic_error: 0.4634
Epoch 91/100
25/25 [==============================] - 0s 959us/step - loss: 5.5748 - mse: 5.5748 - mae: 0.8244 - root_mean_squared_error: 2.3611 - mean_squared_logarithmic_error: 0.4633
Epoch 92/100
25/25 [==============================] - 0s 1ms/step - loss: 5.5725 - mse: 5.5725 - mae: 0.8242 - root_mean_squared_error: 2.3606 - mean_squared_logarithmic_error: 0.4633
Epoch 93/100
25/25 [==============================] - 0s 1ms/step - loss: 5.5702 - mse:

In [17]:
%tensorboard --logdir '/home/bavanya/Documents/Training_curves/cnn_propv85/'

In [18]:
total_training_time

[3.7905354499816895,
 3.6928391456604004,
 3.583998441696167,
 3.558032274246216,
 3.545685052871704,
 3.5432820320129395,
 3.5481653213500977,
 3.52325701713562,
 3.637256622314453,
 3.5552332401275635,
 3.5806474685668945,
 3.5199966430664062,
 3.6791484355926514,
 3.6423346996307373,
 3.5984835624694824,
 3.690652370452881,
 3.656158685684204,
 3.945202350616455,
 3.7117247581481934,
 3.9339072704315186,
 3.7971577644348145,
 3.654139518737793,
 3.7539265155792236,
 3.717947244644165,
 3.5889856815338135,
 3.7296924591064453,
 3.2254416942596436,
 3.2676196098327637,
 3.156820058822632,
 3.2367966175079346]

In [19]:
print("avg training time: " + str(sum(total_training_time)/30))

avg training time: 3.6021689335505167


In [20]:
total_training_loss_history

[{'loss': [20727.837890625,
   17949.13671875,
   15661.0693359375,
   13645.5556640625,
   11973.6669921875,
   10491.763671875,
   9244.666015625,
   8146.60546875,
   7162.25830078125,
   6320.89404296875,
   5577.357421875,
   4993.041015625,
   4523.5576171875,
   4081.885498046875,
   3728.31982421875,
   3350.07568359375,
   3047.239013671875,
   2768.52099609375,
   2514.408935546875,
   2280.791015625,
   2072.93896484375,
   1891.18505859375,
   1723.6856689453125,
   1576.0810546875,
   1435.070556640625,
   1314.3695068359375,
   1210.5736083984375,
   1103.810546875,
   1012.39111328125,
   933.197265625,
   861.430419921875,
   791.5279541015625,
   734.41796875,
   681.668212890625,
   630.6348876953125,
   586.7677001953125,
   546.1315307617188,
   508.0711669921875,
   474.376220703125,
   444.0710144042969,
   415.6287841796875,
   391.0535583496094,
   364.2170104980469,
   344.82794189453125,
   322.5531921386719,
   304.13897705078125,
   286.4594421386719,
   270

In [21]:
sum_mse = 0
sum_mae = 0
training_mse = []
training_mae = []
for i in range(30):
    sum_mse += float(total_training_loss_history[i]['mse'][-1])
    sum_mae += float(total_training_loss_history[i]['mae'][-1])
    training_mse.append(total_training_loss_history[i]['mse'][-1])
    training_mae.append(total_training_loss_history[i]['mae'][-1])
print("avg mse of training: " + str(sum_mse/30))
print("avg mae of training: " + str(sum_mae/30))

avg mse of training: 6.796566597620646
avg mae of training: 0.840122793118159


In [22]:
total_scores

[[6.191804885864258,
  6.191804885864258,
  0.7943955659866333,
  2.4883337020874023,
  0.42581430077552795],
 [4.107142925262451,
  4.107142925262451,
  0.7175324559211731,
  2.02660870552063,
  0.4072096049785614],
 [4.1074395179748535,
  4.1074395179748535,
  0.7192728519439697,
  2.026681900024414,
  0.4074612259864807],
 [5.947223663330078,
  5.947223663330078,
  0.8979594111442566,
  2.438693046569824,
  0.4786958694458008],
 [4.107142925262451,
  4.107142925262451,
  0.7175324559211731,
  2.02660870552063,
  0.4072096049785614],
 [4.107142925262451,
  4.107142925262451,
  0.7175324559211731,
  2.02660870552063,
  0.4072096049785614],
 [4.107142925262451,
  4.107142925262451,
  0.7175324559211731,
  2.02660870552063,
  0.4072096049785614],
 [4.116880893707275,
  4.116880893707275,
  0.7215084433555603,
  2.0290098190307617,
  0.4096982479095459],
 [4.137706756591797,
  4.137706756591797,
  0.7534408569335938,
  2.034135341644287,
  0.41007688641548157],
 [4.107142925262451,
  4.1

In [4]:
a = [[6.191804885864258,
  6.191804885864258,
  0.7943955659866333,
  2.4883337020874023,
  0.42581430077552795],
 [4.107142925262451,
  4.107142925262451,
  0.7175324559211731,
  2.02660870552063,
  0.4072096049785614],
 [4.1074395179748535,
  4.1074395179748535,
  0.7192728519439697,
  2.026681900024414,
  0.4074612259864807],
 [5.947223663330078,
  5.947223663330078,
  0.8979594111442566,
  2.438693046569824,
  0.4786958694458008],
 [4.107142925262451,
  4.107142925262451,
  0.7175324559211731,
  2.02660870552063,
  0.4072096049785614],
 [4.107142925262451,
  4.107142925262451,
  0.7175324559211731,
  2.02660870552063,
  0.4072096049785614],
 [4.107142925262451,
  4.107142925262451,
  0.7175324559211731,
  2.02660870552063,
  0.4072096049785614],
 [4.116880893707275,
  4.116880893707275,
  0.7215084433555603,
  2.0290098190307617,
  0.4096982479095459],
 [4.137706756591797,
  4.137706756591797,
  0.7534408569335938,
  2.034135341644287,
  0.41007688641548157],
 [4.107142925262451,
  4.107142925262451,
  0.7175324559211731,
  2.02660870552063,
  0.4072096049785614],
 [4.607697486877441,
  4.607697486877441,
  0.7520288825035095,
  2.146554708480835,
  0.41090863943099976],
 [4.107142925262451,
  4.107142925262451,
  0.7175324559211731,
  2.02660870552063,
  0.4072096049785614],
 [5.424398899078369,
  5.424398899078369,
  0.7703642249107361,
  2.329033851623535,
  0.42167171835899353],
 [4.766801357269287,
  4.766801357269287,
  0.7768146395683289,
  2.183300495147705,
  0.41825148463249207],
 [4.378990173339844,
  4.378990173339844,
  0.7615808844566345,
  2.0926036834716797,
  0.42509451508522034],
 [4.107142925262451,
  4.107142925262451,
  0.7175324559211731,
  2.02660870552063,
  0.4072096049785614],
 [3.4508540630340576,
  3.4508540630340576,
  0.7825890779495239,
  1.85764741897583,
  0.34054267406463623],
 [4.107142925262451,
  4.107142925262451,
  0.7175324559211731,
  2.02660870552063,
  0.4072096049785614],
 [4.165094375610352,
  4.165094375610352,
  0.730279266834259,
  2.04085636138916,
  0.41462618112564087],
 [4.35645055770874,
  4.35645055770874,
  0.7661636471748352,
  2.0872111320495605,
  0.4051661193370819],
 [4.753446578979492,
  4.753446578979492,
  0.7904454469680786,
  2.1802399158477783,
  0.42747706174850464],
 [4.062610626220703,
  4.062610626220703,
  0.7126678228378296,
  2.015591859817505,
  0.4011302590370178],
 [6.073475360870361,
  6.073475360870361,
  0.7885535359382629,
  2.464442253112793,
  0.42427217960357666],
 [4.107142925262451,
  4.107142925262451,
  0.7175324559211731,
  2.02660870552063,
  0.4072096049785614],
 [4.1705121994018555,
  4.1705121994018555,
  0.7462447285652161,
  2.0421831607818604,
  0.3924141228199005],
 [11.133997917175293,
  11.133997917175293,
  0.8496005535125732,
  3.3367645740509033,
  0.43238314986228943],
 [4.092278957366943,
  4.092278957366943,
  0.7243729829788208,
  2.0229382514953613,
  0.40276476740837097],
 [4.301105499267578,
  4.301105499267578,
  0.734700620174408,
  2.073910713195801,
  0.4161568284034729],
 [4.181606769561768,
  4.181606769561768,
  0.7644259929656982,
  2.0448977947235107,
  0.4260184168815613],
 [4.124817848205566,
  4.124817848205566,
  0.7235491275787354,
  2.0309648513793945,
  0.40899911522865295]]

In [5]:
test_mae = []
for i in range(10):
    test_mae.append(a[i][2])
sum(test_mae)/10

0.7474239408969879

In [3]:
test_mae

[0.7943955659866333,
 0.7175324559211731,
 0.7192728519439697,
 0.8979594111442566,
 0.7175324559211731,
 0.7175324559211731,
 0.7175324559211731,
 0.7215084433555603,
 0.7534408569335938,
 0.7175324559211731,
 0.7520288825035095,
 0.7175324559211731,
 0.7703642249107361,
 0.7768146395683289,
 0.7615808844566345,
 0.7175324559211731,
 0.7825890779495239,
 0.7175324559211731,
 0.730279266834259,
 0.7661636471748352,
 0.7904454469680786,
 0.7126678228378296,
 0.7885535359382629,
 0.7175324559211731,
 0.7462447285652161,
 0.8496005535125732,
 0.7243729829788208,
 0.734700620174408,
 0.7644259929656982,
 0.7235491275787354]

In [26]:
sum_mse = 0
sum_mae = 0
sum_are = 0
test_mse = []
test_mae = []
for i in range(30):
    sum_mse += float(total_scores[i][0])
    sum_mae += float(total_scores[i][2])
    test_mse.append(total_scores[i][0])
    test_mae.append(total_scores[i][2])
    sum_are += float(test_ARE[i])
print("avg mse: " + str(sum_mse/30))
print("avg mae: " + str(sum_mae/30))
print("avg are: " + str(sum_are/30))

avg mse: 4.6503160238265995
avg mae: 0.7506250222524007
avg are: 0.31027364134788515


In [27]:
print("median training mse: " + str(statistics.median(training_mse)))
print("median testing mse: " + str(statistics.median(test_mse)))

median training mse: 5.51245903968811
median testing mse: 4.131262302398682


In [28]:
print("median training mae: " + str(statistics.median(training_mae)))
print("median testing mae: " + str(statistics.median(test_mae)))

median training mae: 0.8199346363544464
median testing mae: 0.7324899435043335


In [29]:
print("median testing ARE: " + str(statistics.median(test_ARE)))

median testing ARE: 0.3100649416446686


In [30]:
test_ARE

[0.3212121,
 0.31006494,
 0.31006494,
 0.31577381,
 0.31006494,
 0.31006494,
 0.31006494,
 0.31006494,
 0.3086039,
 0.31006494,
 0.30814397,
 0.31006494,
 0.3116883,
 0.30641234,
 0.30925325,
 0.31006494,
 0.30071452,
 0.31006494,
 0.31006494,
 0.30205628,
 0.3196158,
 0.30762988,
 0.3146104,
 0.31006494,
 0.30565473,
 0.31709957,
 0.31006494,
 0.30876622,
 0.31006494,
 0.31006494]

In [6]:
sum([0.3212121,
 0.31006494,
 0.31006494,
 0.31577381,
 0.31006494,
 0.31006494,
 0.31006494,
 0.31006494,
 0.3086039,
 0.31006494,
 0.30814397,
 0.31006494,
 0.3116883,
 0.30641234,
 0.30925325,
 0.31006494,
 0.30071452,
 0.31006494,
 0.31006494,
 0.30205628,
 0.3196158,
 0.30762988,
 0.3146104,
 0.31006494,
 0.30565473,
 0.31709957,
 0.31006494,
 0.30876622,
 0.31006494,
 0.31006494][:10])/10

0.311604439